In [12]:
import csv
import json
import pandas as pd
from datetime import datetime
import numpy as np

from collections import Counter

outputFolders = "../data_2019_processed/"

input_file_sufix = "_filtred.csv"
fileOutPutSufixName = "_transformed.csv"

In [13]:
def convert_to_decibels(value):
    if(value > 0.0):
        return 20 * (np.log(32768) - np.log(value/32768) )
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0

def treat_data_to_0_1(value):
    if(value > 0.0):
        return 1.0
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0
    
def transform_data(dataframe):
    # convert data and show the new info
    # activity
    dataframe['activity'] = dataframe['activity'].replace(['Unknown'], '1')
    dataframe['activity'] = dataframe['activity'].replace(['Still'], '2')
    dataframe['activity'] = dataframe['activity'].replace(['Tilting'], '3')
    dataframe['activity'] = dataframe['activity'].replace(['Exercise'], '4')
    dataframe['activity'] = dataframe['activity'].replace(['In vehicle'], '5')
    # class
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    # day_of_week
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Tuesday'], '1')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Wednesday'], '2')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Thursday'], '3')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Friday'], '4')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Saturday'], '5')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Sunday'], '6')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Monday'], '7')
    # phone_lock
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['not locked'], '0')
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['locked'], '1')
    # location
    dataframe['location'] = dataframe['location'].replace(['Other'], '1')
    dataframe['location'] = dataframe['location'].replace(['University'], '2')
    dataframe['location'] = dataframe['location'].replace(['House'], '3')
    
    # timestamp
    if 'timestamp2' in dataframe.columns:
        dataframe['timestamp_text'] = dataframe['timestamp']
        dataframe['timestamp'] = dataframe['timestamp2']
        del dataframe['timestamp2']
    if 'outside_range' in dataframe.columns:
        del dataframe['outside_range']
    if 'time_sleeping' in dataframe.columns:
        del dataframe['time_sleeping']
    if 'wakeup_time' in dataframe.columns:
        del dataframe['wakeup_time']
    if 'sleep_time' in dataframe.columns:
        del dataframe['sleep_time']
    if 'distanceHome' in dataframe.columns:
        del dataframe['distanceHome']    
    if 'showActuation' in dataframe.columns:
        del dataframe['showActuation'] 
    #dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[ns]')
    #dataframe['timestamp'] = dataframe['timestamp'].astype('int64')
    #dataframe['timestamp'] = datetime.strptime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S').timestamp() * 1000
    #dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S') * 1000
    #dataframe['timestamp'] = dataframe['timestamp'].dt.second()
    # data_type
    #dataframe['activity'] = dataframe['activity'].astype(np.integer) 
    #dataframe['class'] = dataframe['class'].astype(np.integer) 
    #dataframe['day_of_week'] = dataframe['day_of_week'].astype(np.integer) 
    #dataframe['phone_lock'] = dataframe['phone_lock'].astype(np.integer) 
    #dataframe['location'] = dataframe['location'].astype(np.integer) 
    dataframe['sound'] = dataframe['sound'].apply(convert_to_decibels)
    dataframe['proximity'] = dataframe['proximity'].apply(treat_data_to_0_1)
    dataframe['activity'] = dataframe['activity'].astype('int32')
    dataframe['class'] = dataframe['class'].astype('int32')
    dataframe['day_of_week'] = dataframe['day_of_week'].astype('int32')
    dataframe['phone_lock'] = dataframe['phone_lock'].astype('int32') 
    dataframe['location'] = dataframe['location'].astype('int32')
    #dataframe['minutes_day'] = dataframe['minutes_day'].astype('float32')
    dataframe['time_to_next_alarm'] = dataframe['time_to_next_alarm'].astype('float32')
    dataframe['sound'] = dataframe['sound'].astype('float32')
    dataframe['light'] = dataframe['light'].astype('float32')
    dataframe['proximity'] = dataframe['proximity'].astype('float32')
    dataframe['timestamp'] = dataframe['timestamp'].astype('float32')
    
    temp = dataframe['class'].copy()
    del dataframe['class']
    dataframe['class'] = temp
    #return dataframe
    return dataframe

In [14]:
def transform_normalize_data(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name != "class" and feature_name != "timestamp_text":
            max_value = df[feature_name].max()
            if(feature_name == "timestamp"):
                max_value = 1924905600.0 #2030-12-31 00:00:00  
            if(feature_name == "day_of_week"):
                max_value = 7            
            if(feature_name == "sound"):
                max_value = 403 # overhal higher value            
            if(feature_name == "location"):
                max_value = 3
            if(feature_name == "activity"):
                max_value = 5
            if(feature_name == "minutes_day"):
                max_value = 1439 # 24 horas
            if(feature_name == "time_to_next_alarm"):
                max_value = 604800000.0 # 2.8 years in ms
            min_value = df[feature_name].min()
            print(feature_name+" "+str(min_value)+" "+str(max_value))
            if feature_name != "showActuation":
                result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
            if(max_value == 0 and min_value == 0):
                result[feature_name] = 0.0
    return result

In [15]:
def transform_data_type(dataset):
    # diminui o tamanho do tipo de dado (float32 é mais que o suficiente)
    dataset['activity'] = dataset['activity'].astype('int32')
    dataset['location'] = dataset['location'].astype('int32')
    dataset['timestamp'] = dataset['timestamp'].astype('float32')
    dataset['day_of_week'] = dataset['day_of_week'].astype('int32')
    dataset['light'] = dataset['light'].astype('int32')
    dataset['phone_lock'] = dataset['phone_lock'].astype('int32')
    dataset['proximity'] = dataset['proximity'].astype('int32')
    dataset['sound'] = dataset['sound'].astype('float32')
    dataset['time_to_next_alarm'] = dataset['time_to_next_alarm'].astype('float32')
    
    return dataset

In [16]:
def transform_output_to_numeric(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    dataframe['class'] = dataframe['class'].astype('int32')
    
    return dataframe

In [17]:
def transform_output_to_str(dataframe):
    dataframe['class'] = dataframe['class'].apply(str)
    #print(dataframe['class'])
    dataframe['class'] = dataframe['class'].replace(to_replace='0',value='awake')
    dataframe['class'] = dataframe['class'].replace(to_replace='1',value='asleep')
    
    return dataframe

# Files by students

In [18]:
# Dataframe of all datasets
# initialize list of lists
datasets = [['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs'],
            ['0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA'], 
            ['2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0'], 
            ['2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys'], 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            ['7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA'], 
            ['a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4'], 
            ['ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc'], 
            ['Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U'], 
            ['CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA'], 
            ['DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc'], 
            ['DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA'], 
            ['dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY'], 
            ['HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo'], 
            ['jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw'], 
            ['JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I'], 
            ['K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8'], 
            ['oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q'], 
            ['pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM'], 
            ['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            ['QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ'], 
            ['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            ['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            ['SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'], 
            ['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            ['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            ['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            ['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            ['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            ['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]
  
# Create the pandas DataFrame
folders = pd.DataFrame(datasets, columns=['folder'])

#folders

In [19]:
pd.read_csv(outputFolders+"student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv")


,activity,location,distanceHome,showActuation,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,sleep_time,wakeup_time,time_sleeping,timestamp_text,class
0,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,1020,660,64808,2018-05-15 14:20:45+00:00,awake
1,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,1020,660,64808,2018-05-15 14:20:45+00:00,awake
2,Still,University,0,False,1526390475,593940000.0,5517,5,not locked,35,Tuesday,861,1020,660,64808,2018-05-15 14:21:15+00:00,awake
3,Still,University,0,False,1526390505,593940000.0,5517,5,not locked,307,Tuesday,861,1020,660,64808,2018-05-15 14:21:45+00:00,awake
4,Still,University,0,False,1526390535,593880000.0,5809,5,not locked,42,Tuesday,862,1020,660,64808,2018-05-15 14:22:15+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17988,Still,House,0,False,1528892024,594720000.0,7974,0,locked,0,Wednesday,793,0,0,0,2018-06-13 13:13:44+00:00,awake
17989,Unknown,House,0,False,1528892094,594660000.0,10516,0,locked,0,Wednesday,794,0,0,0,2018-06-13 13:14:54+00:00,awake
17990,Exercise,House,0,False,1528892144,594600000.0,10516,0,locked,0,Wednesday,795,0,0,0,2018-06-13 13:15:44+00:00,awake
17991,Unknown,House,0,False,1528892175,594600000.0,4438,0,locked,0,Wednesday,796,0,0,0,2018-06-13 13:16:15+00:00,awake


In [21]:
pd.read_csv(outputFolders+"student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs"+input_file_sufix)


,activity,location,distanceHome,showActuation,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class,outside_range
0,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,2018-05-15 14:20:45+00:00,awake,0
1,Unknown,Other,0,False,1526390445,594000000.0,0,5,not locked,37,Tuesday,860,2018-05-15 14:20:45+00:00,awake,0
2,Still,University,0,False,1526390475,593940000.0,5517,5,not locked,35,Tuesday,861,2018-05-15 14:21:15+00:00,awake,0
3,Still,University,0,False,1526390505,593940000.0,5517,5,not locked,307,Tuesday,861,2018-05-15 14:21:45+00:00,awake,0
4,Still,University,0,False,1526390535,593880000.0,5809,5,not locked,42,Tuesday,862,2018-05-15 14:22:15+00:00,awake,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17348,Still,House,0,False,1528857908,545760000.0,94,5,locked,3,Tuesday,225,2018-06-13 03:45:08+00:00,awake,0
17349,Still,House,0,False,1528857938,545700000.0,740,5,locked,3,Tuesday,225,2018-06-13 03:45:38+00:00,awake,0
17350,Still,House,0,False,1528857996,545700000.0,740,5,locked,3,Tuesday,226,2018-06-13 03:46:36+00:00,awake,0
17351,Still,House,0,False,1528858461,545580000.0,257,5,locked,3,Tuesday,234,2018-06-13 03:54:21+00:00,awake,0


In [22]:
for indexFolder, folder in folders.iterrows():
    dataset = pd.read_csv(outputFolders+"student_"+folder['folder']+input_file_sufix)
    print()
    print("student_"+str(folder['folder'])+".csv", Counter(dataset['class']) )
    # basic transformations 
    datasetNumericClassification = transform_data(dataset)
    datasetNumericClassification = transform_data_type(datasetNumericClassification)
    datasetNumericClassificationNormalized = transform_normalize_data(datasetNumericClassification)
    datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericClassificationNormalized)

    datasetNumericClassificationNormalized.to_csv(outputFolders+"student_"+folder['folder']+fileOutPutSufixName, sep=',', encoding='utf-8', index=False)


student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv Counter({'awake': 15701, 'asleep': 1652})
activity 1 5
location 1 3
timestamp 1526390400.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0 1
phone_lock 0 1
light 0 60000
day_of_week 1 7
minutes_day 0 1439

student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA.csv Counter({'awake': 4914, 'asleep': 3403})
activity 1 5
location 1 3
timestamp 1526415200.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0 0
phone_lock 0 1
light 0 0
day_of_week 1 7
minutes_day 0 1439

student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0.csv Counter({'asleep': 1319, 'awake': 1124})
activity 1 5
location 1 3
timestamp 1526390500.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0 0
phone_lock 0 1
light 0 0
day_of_week 1 7
minutes_day 8 1439

student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys.csv Counter({'awake': 8366, 'asleep': 2803})
activity 1 5
location 1 3
timestamp 1526263000.0 1

In [23]:
datasetNumericClassificationNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.974504,0.000000,1.0,0.0,0.004956,0.333333,0.681723,2018-05-17 16:21:34+00:00,awake
1,0.00,0.5,0.000000e+00,0.974405,0.604264,1.0,0.0,0.002154,0.333333,0.682418,2018-05-17 16:22:34+00:00,awake
2,0.25,0.5,3.213375e-07,0.974306,0.597445,1.0,0.0,0.003629,0.333333,0.683113,2018-05-17 16:23:04+00:00,awake
3,0.00,0.5,3.213375e-07,0.974306,0.597445,1.0,0.0,0.002213,0.333333,0.683113,2018-05-17 16:23:34+00:00,awake
4,0.25,0.5,3.213375e-07,0.974206,0.536104,1.0,0.0,0.003157,0.333333,0.683808,2018-05-17 16:24:34+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
13454,0.25,0.5,4.564599e-03,0.968155,0.700926,1.0,1.0,0.003924,0.333333,0.726199,2018-06-07 17:25:51+00:00,awake
13455,0.25,0.5,4.564599e-03,0.968056,0.700926,1.0,1.0,0.003953,0.333333,0.726894,2018-06-07 17:26:21+00:00,awake
13456,0.25,0.5,4.564920e-03,0.968056,0.674788,1.0,1.0,0.004101,0.333333,0.726894,2018-06-07 17:26:51+00:00,awake
13457,0.25,0.5,4.565242e-03,0.967857,0.700675,1.0,1.0,0.005074,0.333333,0.728284,2018-06-07 17:28:51+00:00,awake


In [24]:
print("Test")
pd.read_csv(outputFolders+"student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw"+fileOutPutSufixName)

Test


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.0,0.000000,1.0,0.0,0.00004,0.166667,0.537874,2018-05-16 12:54:23+00:00,awake
1,0.00,0.0,0.000000e+00,0.0,0.000000,1.0,0.0,0.00004,0.166667,0.537874,2018-05-16 12:54:23+00:00,awake
2,0.25,1.0,0.000000e+00,0.0,0.724465,1.0,0.0,0.00005,0.166667,0.537874,2018-05-16 12:54:53+00:00,awake
3,0.25,1.0,3.212578e-07,0.0,0.631813,1.0,0.0,0.00004,0.166667,0.538568,2018-05-16 12:55:53+00:00,awake
4,0.00,1.0,3.212578e-07,0.0,0.631813,1.0,0.0,0.00005,0.166667,0.539263,2018-05-16 12:56:23+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
16070,0.25,1.0,5.718389e-03,0.0,0.810921,0.0,1.0,0.00000,1.000000,0.908965,2018-06-11 21:48:55+00:00,awake
16071,0.50,1.0,5.718710e-03,0.0,0.810921,1.0,0.0,0.00003,1.000000,0.911049,2018-06-11 21:51:04+00:00,awake
16072,0.50,1.0,5.719031e-03,0.0,0.552725,1.0,0.0,0.00003,1.000000,0.911049,2018-06-11 21:51:34+00:00,awake
16073,0.50,1.0,5.719031e-03,0.0,0.552725,1.0,1.0,0.00003,1.000000,0.911744,2018-06-11 21:52:06+00:00,awake
